# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [50]:
%matplotlib inline
from math import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'classificado1.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo classificado1.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
# Colocando as classificações : 0: "Negativo", 1: "Neutro", 2: "Positivo"
train["Classificação"] = train["Classificação"].astype("category")
train["Classificação"].cat.categories = ["Relevante","Irrelevante"]

train.head(5)

,Treinamento,Classificação
0,@anyretro acho que eu amo o de flocos e nutella,Relevante
1,pedi pastel de camarão e sla agr to arrependid...,Relevante
2,algm manda um pote de nutella pra mim que eu f...,Relevante
3,@corinthians_ff torcida modinha time vai mal r...,Irrelevante
4,"@marciom61326104 cai fora paga pau de chines, ...",Irrelevante


In [26]:
test = pd.read_excel(filename, sheet_name = 'Teste')
# Colocando as classificações : 0: "Negativo", 1: "Neutro", 2: "Positivo"
test["Classificação"] = test["Classificação"].astype("category")
test["Classificação"].cat.categories = ["Relevante","Irrelevante"]
test.head(5)

,Teste,Classificação
0,@duvirmond tênis? nutella,Irrelevante
1,como assim tem gente q n come nutella no açaí?...,Relevante
2,"das delícias da abuelita, bolo ferrero! a comb...",Relevante
3,daqui a pouco enfiam nutella e granulado como ...,Relevante
4,biel querendo expulsão da raissa por causa de ...,Irrelevante


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [54]:
#Nesta célula vamos preparar os dados para fazer as probabilidades.



#Twitters do treinamento com suas devidas classificações(Série)
train_r = train.loc[train["Classificação"] == "Relevante",:]
train_ir = train.loc[train["Classificação"] == "Irrelevante",:]

#Twitters do treinamento com suas devidas classificações(Lista)
twitters_r = []
twitters_ir = []
twitters_tudo = []

#Palavras do treinamento com suas devidas classificações(Lista)
palavras_r = []
palavras_ir = []
palavras_tudo = []

# loop para encontrar os twitters
for i in train_r.Treinamento:
    twitters_r.append(i)
    
for i in train_ir.Treinamento:
    twitters_ir.append(i)
    
for i in train.Treinamento:
    twitters_tudo.append(i)

# loop para encontrar as palavras

for i in twitters_r:
    word = i.split()
    for t in word:
         palavras_r.append(t)
    
for i in twitters_ir:
    word = i.split()
    for t in word:
         palavras_ir.append(t)

for i in twitters_tudo:
    word = i.split()
    for t in word:
         palavras_tudo.append(t)

#Função de limpeza de caracteres irrelevantes para o Nayve-Bayes:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@/•\]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
#Funçao que separa emoji de emojis e palavras de emojis:

def emoji(text):
    import emoji
    novo_texto = ''
    for character in text:
        if character in emoji.UNICODE_EMOJI :
            novo_texto += ' '+ character + ' '
        else:
            novo_texto += character
    return novo_texto

# Transformando as palavras em texto:
texto_r = ""
texto_ir = ""
texto_tudo = ""

# o loop para tranformar palavras em texto:
for i in palavras_r:
    texto_r += " "
    texto_r += i

for i in palavras_ir:
    texto_ir += " "
    texto_ir += i
    
for i in palavras_tudo:
    texto_tudo += " "
    texto_tudo += i

#Limpando os textos:
texto_r = emoji(cleanup(texto_r.lower()))
texto_ir = emoji(cleanup(texto_ir.lower()))
texto_tudo = emoji(cleanup(texto_tudo.lower()))

#Colocando tudo em uma série e um lista:
serie_tudo = pd.Series(texto_tudo.split())
serie_r = pd.Series(texto_r.split())
serie_ir = pd.Series(texto_ir.split())

lista_tudo = list(texto_tudo.split())
lista_r = list(texto_r.split())
lista_ir = list(texto_ir.split())





error: unterminated character set at position 0

In [55]:
#Vamos definir os priors
prior_r = len(twitters_r)/len(twitters_tudo)
prior_ir = len(twitters_ir)/len(twitters_tudo)
# Vamos começar a construir as probabilidades
prob_tudo = serie_tudo.value_counts()
prob_r = serie_r.value_counts()
prob_ir = serie_ir.value_counts()

# Quantidade de palavras em cada classificação
T_tudo = len(serie_tudo)
T_r= len(serie_r)
T_ir = len(serie_ir)

#Uma vez construida as probabilidades podemos começar a pensar na função para a classificação:
#Mas precisamos um alfa e um N para fazer o Laplace smoothing:
#alpha(a):1
#N(N): 10^6

def classifica(frase):
    a = 1
    N = 10**6
    p_r = log(prior_r,10)
    p_ir = log(prior_ir,10)
    palavras = frase.split()
    for i in palavras:
        
        if i not in lista_r:
            p_r += log(a/(T_r+N*a),10)
        elif i in lista_r:
            p_r += log(a+prob_r[i]/(T_r+N*a),10)
            
        if i not in lista_ir:
            p_ir += log(a/(T_ir+N*a),10)
        elif i in lista_ir:
            p_ir += log(a+prob_ir[i]/(T_ir+N*a),10)
            
    if p_r > p_ir:
        return "Relevante"
    else:
        return "Irrelevante"

In [57]:
l

['anyretro',
 'acho',
 'que',
 'eu',
 'amo',
 'o',
 'de',
 'flocos',
 'e',
 'nutella',
 'pedi',
 'pastel',
 'de',
 'camarão',
 'e',
 'sla',
 'agr',
 'to',
 'arrependida',
 'de',
 'n',
 'ter',
 'pedido',
 'de',
 'nutella',
 'c',
 'morango',
 '😭',
 '😭',
 '😭',
 '😭',
 'algm',
 'manda',
 'um',
 'pote',
 'de',
 'nutella',
 'pra',
 'mim',
 'que',
 'eu',
 'falo',
 'se',
 'esperava',
 'ou',
 'não',
 '👉',
 '🏼',
 '👈',
 '🏼',
 'já',
 'vi',
 'de',
 'tudo',
 'nessa',
 'vida',
 'mas',
 'batata',
 'frita',
 'com',
 'nutella',
 'não',
 'entra',
 'puta',
 'que',
 'pariu',
 'fui',
 'brincar',
 'com',
 'a',
 'nutella',
 'e',
 'ela',
 'sujou',
 'tudinho',
 'minha',
 'calça',
 'mano',
 'rt',
 'pqpjulhaa',
 'td',
 'que',
 'eu',
 'queria',
 'era',
 'um',
 'açaí',
 'c',
 'nutella',
 'karinanakahara7',
 'eu',
 'vi',
 'um',
 'de',
 'nutella',
 'e',
 'morango',
 'fiquei',
 '🤪',
 '🤤',
 'leite',
 'condensado',
 'cozido',
 'de',
 'sobremesa',
 'pq',
 'sim',
 'e',
 'nutella',
 'é',
 'superestimada',
 'pás',
 'seulcudi

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [46]:
# Separando o dataframe somente na parte com os twitters:
test_ = test.Teste

#Adicionando os Twitters em uma lista:
test_twitters = test_.tolist()


            

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**